In [ ]:
import numpy as np

In [ ]:
# My pricer, not fully working
def BOPM(vol, r, T, K, S0, n, print_array=False, call=True):
  dt = T/n
  u = np.exp(vol * np.sqrt(dt))
  d = u ** - 1
  p = (np.exp(u*dt)-d)/(u-d)

  # Price tree 
  tree = np.zeros([n+1, n+1])

  for i in range(n+1):
    for j in range(i+1):
      tree[j, i] = S0*(d**j)*(u**(i-j))
  
  # Option value
  option = np.zeros([n+1, n+1])
  if call:
    option[:, n] = np.maximum(np.zeros(n+1), tree[:, n]-K)
  else:
    option[:, n] = np.maximum(np.zeros(n+1), K-tree[:, n])



  # Option price at t=0
  for i in np.arange(n-1, -1, -1):
    for j in np.arange(0, i+1):
      option[j, i] = np.exp(-r*dt)*(p*option[j, i+1] + (1-p)*option[j+1, i+1])

  if print_array:
    return "Price at t=0: ${}".format(round(option[0, 0], 4)), tree, option, "Max payoff = ${}".format(round(np.maximum(option[j, n], 4), 0))
  else:
    return "Price at t=0: ${}".format(round(option[0, 0], 4)), "Max payoff = ${}".format(np.maximum(round(option[j, n], 4), 0))


In [1]:
# Print tree
import numpy as np
import math as m
import timeit

def bop(n,t,S,v):
    dt = t/n
    u = m.exp(v*m.sqrt(dt))
    d = 1/u
    Pm = np.zeros((n+1, n+1))
    for j in range(n+1):
        for i in range(j+1):
            Pm[i,j] = S*m.pow(d,i) * m.pow(u,j-i)
    return Pm

In [ ]:
# Test tree print
n = 5
t = 1/2
S = 102
v = 0.409
x = bop(n,t,S,v)


print('n = 1:\n',np.matrix(x.astype(int)))

In [ ]:
# Pure Cython BOPM with Hedge ratio calculator
"""Pure Python binomial options pricing using a Jarrow-Rudd model.
To use this interactively, just do::
    >>> from binomial import jarrow_rudd
    >>> jarrow_rudd(100.0, 100.0, 1.0, 0.3, 0.03, -1, True, 100)
    10.627682380924412
"""
import numpy as np
import math

def jarrow_rudd(s, k, t, v, rf, cp, am=False, n=100):
    """Price an option using the Jarrow-Rudd binomial model.
    
    s : initial stock price
    k : strike price
    t : expiration time
    v : volatility
    rf : risk-free rate
    cp : +1/-1 for call/put
    am : True/False for American/European
    n : binomial steps
    """
    #Basic calculations
    h = t/n
    u = math.exp((rf-0.5*math.pow(v,2))*h+v*math.sqrt(h))
    d = math.exp((rf-0.5*math.pow(v,2))*h-v*math.sqrt(h))
    drift = math.exp(rf*h)
    q = (drift-d)/(u-d)
    
    # Hedge Ratio
    cu = np.maximum((u * s) - k, 0)
    cd = np.maximum((d * s) - k, 0)
    m = ((cu - cd)/((u - d) * s)) - 1

    #Process the terminal stock price
    stkval = np.zeros((n+1,n+1))
    optval = np.zeros((n+1,n+1))
    stkval[0,0] = s
    for i in range(1,n+1):
        stkval[i,0] = stkval[i-1,0]*u
        for j in range(1,i+1):
            stkval[i,j] = stkval[i-1,j-1]*d

    #Backward recursion for option price
    for j in range(n+1):
        optval[n,j] = max(0,cp*(stkval[n,j]-k))
    for i in range(n-1,-1,-1):
        for j in range(i+1):
            optval[i,j] = (q*optval[i+1,j]+(1-q)*optval[i+1,j+1])/drift
            if am:
                optval[i,j] = max(optval[i,j],cp*(stkval[i,j]-k))

    print('Option Price: ', round(optval[0, 0], 3))
    print('Hedge Ratio: ', round(m, 4))
    
    return stkval.shape


In [ ]:
jarrow_rudd(50, 52, 0.5, 0.15, 0.06, 1, False)

In [ ]:
def hedge(u, d, s, k):
  cu = np.maximum((u * s) - k, 0)
  cd = np.maximum((d * s) - k, 0)
  m = ((cu - cd)/((u - d) * s)) - 1

  return m

In [ ]:
hedge(1.05, 0.95, 72, 68)

In [ ]:
jarrow_rudd(100, 95, 1, 0.2, 0.07, -1, True, 12)

In [ ]:
# Exotic pricer
def exotic_option_pricer(s0, k, u, d, r, T, cp):
  """Price an exotic option using the binomial model.
    
    s0 : initial stock price
    k : strike price ** 2
    u : upfactor
    d : downfactor
    r : risk-free rate
    cp : +1/-1 for call/put
    """
  
  # Prefactors
  eu = np.maximum((((u * s0) ** 2) - k), 0)
  ed = np.maximum((((d * s0) ** 2) - k), 0)

  # m shares and b mma
  m = (eu - ed) / ((u * s0) - (d * s0))
  b = (((u * ed) - (d * eu)) / (u - d)) * np.exp(-r * T)
  v = m * s0 + b


  return v

exotic_option_pricer(50, 2500, 1.16038286, 0.85963276, 0.04, 0.25, 1)
# exotic_option_pricer(100, 10000, 1.2, 0.9025, 0.05, 1, 1)

In [ ]:
exotic_option_pricer(100, 10, 1.2, 0.8, 0.02, 0.25, 1)